In [ ]:
import collections
import random
import gym
import numpy as np

In [1]:
from cartPoleDqn import DQN

In [2]:
class Agent:
    def __init__(self):
        pass

    def get_action(self):
        pass

    def train(self):
        pass

    def remember(self):
        pass
    
    def replay(self):
        pass

    def play(self):
        pass

IndentationError: expected an indented block (1932603921.py, line 1)